# Assignment 6
## Spike train analysis

<div class="alert alert-block alert-warning">
<b>Preparing for this assignment:</b> 

These resources will help you prepare for this assignment. Watch:
<ul>
        <li> <a href="https://youtu.be/smHwRzk81b0?t=688" target="_blank">this video</a> (until 1:01'54) from a MIT course by Prof. Michale Fee to learn about key concepts for spike train analysis: the <b>Poisson process</b>, <b>Fano Factor</b>, <b>inter-spike intervals</b>, <b>cross-correlation</b> and <b>auto-correlation</b>. You do NOT need to follow the equations, just the concepts. </li> 
        <li> <a href="https://youtu.be/m1w7oywzwpA" target="_blank">the first part of this video</a> (until 17'15) to learn about Generalized Linear Models (GLM) and in particular about the <b>Poisson GLM for spike count regression</b>. Things get a bit more mathy after 10'12: again, forget about the equations if you cannot follow them and try to focus on the concepts. Remember that we have used GLMs previously: linear regression and logistic regressions are classes of GLMs, for continuous and binary data respectively.</li>
</ul>
</div>

# 0. Introduction
In this assignment we will learn about tools to visualize, analyze and model spiking data. Spike trains are different from typical type series such as EEG or fMRI data as they consist of the list of timings for discrete events (action potentials). The same tools can be applied to analyze other event data such as eye saccades, or lever presses in animal conditioning experiments.

The data will be used are spiking activity collected from **one neuron of the dorso-medial striatum** (a structure of the basal ganglia) of one rat while it performs a **perceptual decision-making task**. In one of the analysis we will also look at another neuron recorded during the same experimental session. The (complex) details of the behavioral protocol are available in this [publication](https://www.nature.com/articles/s41467-020-14824-w), but for all we care here **the animal basically has to go to a left or right port depending on whether the dominant tone in an acoustic stimuli is high frequency or low frequency**. The difficulty of the trial is manipulated in a variable called "stimulus evidence" (or simply Stimulus in the csv file), bounded between -1 and 1. Its value is +1 for pure high-frequency stimulus (i.e. clear evidence towards the rightward response), -1 for pure low-frequency stimulus  (i.e. clear evidence towards the leftward response), and values in between for a mix of low-frequency and high-frequency stimulus (for example stimulus 0 means no evidence at all for either response).

Our goal here is to study the basic characteristics of this neuron spiking activity and **whether the neuron activity encodes for the stimulus information and/or the response of the animal**.

Let us first import the typical packages (plus some others we will need along the way).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import expon, norm # for exponential and gaussian distributions
from scipy.signal import correlate, correlation_lags # for auto-correlation / cross-correlation

Now we will load the spiking data and the behavioral data. The spiking data will be loaded as a simple numpy array which represents the timing (in seconds) of all spikes for this neuron during this session. The behavioral data is loaded as a dataframe, comprising the following variables: `Stimulus` (stimulus evidence), `Response` (1: left; 2:right), `Outcome` (0: incorrect, 1:correct), `tResponse` (the timing of the onset of response, in seconds). This last variable will help us isolate the neuron spikes that occur around each trial.

In [ ]:
# load spiking data as numpy array
SpikeTimes =  np.loadtxt("https://raw.githubusercontent.com/wimmerlab/MBC-DataAnalysis/main/A4_SpikeTrainAnalysis/Neuron1_Spike.csv")

# load behavioral data as dataframe
df = pd.read_csv("https://raw.githubusercontent.com/wimmerlab/MBC-DataAnalysis/main/A4_SpikeTrainAnalysis/BehavioralData.csv")
df.head()

# 1. Basic measures
## 1.1 Displaying spikes

**How many spikes did we record in total for this neuron?**

In [ ]:
??

**Let's plot the spike times in a certain window**, to get a rough idea of how this neuron fires.
Adjust the window to look at the activity at different time scales.

In [ ]:
# Define window (start time - end time)
win = ??

# Select all spikes within this window
SpikesInWindow = ??

# plot as vertical ticks
plt.plot(???);

# add labels
plt.xlabel('time (s)');
plt.yticks([]);

## 1.2 Computing spike rate
**Compute the average spike rate of this neuron**, i.e. the total number of spikes divided by the length of the time window (in seconds).

In [ ]:
# length of window
WinLength = ??

# spike rate
SpikeRate = ??

print(??)

## 1.3 Distribution of inter-spike intervals
**Plot the distribution of inter-spike intervals** (i.e. the time intervals between two successive spikes). Hints: use the function `np.diff`; to plot a distribution, use the function `plt.hist` and set the parameter `density` to `True`.

We know that if the neuron spikes according to an homogeneous Poisson process, this distribution should be exponential (with a scale equal to the inverse of the neuron spike rate).
**Plot the prediction for the ISI distribution for the Poisson process on top of the histogram.**

In [ ]:
# compute the array inter-spike intervals
ISI = ??

# plot the experimental distribution of ISI (adjust the number of bins)
plt.hist(???, label = 'experimental')

# scale of the Poisson process
scale =??

# array of ISI values for which we want to compute the predicted pdf
ISIrange = np.arange(??)

# corresponding values for probability distribution
p_Poisson = expon.pdf(ISIrange, scale=scale)

# plot as red curve
plt.plot(??, label='Poisson'); 

# add label
plt.xlabel(??);
plt.ylabel(??);
plt.legend;


**Interpret**

# 2. Auto-correlation and cross-correlation

## 2.1 Auto-correlation

We want to understand better the dynamics of spiking activity, using the tool of auto-correlation. As explained in the first video, auto-correlation tells us about the **time scales that regulate spiking activity**: whether there are periods of higher firing (due to either external factors, i.e. the presence of a stimulus to which the neuron responds; or internal factors, i.e. increased excitability in the local neural network), and the time scales of this period. Remember: the auto-correlation of the (really dull) homogeneous Poisson process is completely flat at 0 (except for the value at zero-lag, as a variable is always correlated with itself).

Auto-correlation (or cross-correlation) tools do not work directly on spiking data but on discrete time series (like EEG,...). So our first step is to "bin spikes", i.e. define regular time bins of a certain duration and compute the number of spikes in each bin. **Bin the neuron spikes with a time resolution of 10 ms** (i.e. time bin duration = 10 ms). Hint: use `np.histogram`.

In [ ]:
# define time bin duration (in SECONDS!!)
dt = ??

# define our time grid, i.e. an array of values spanning all the recording time interspersed with dt
time_grid = np.arange(??)

# compute the number of spikes in each bin
SpikeCount = ???

# display first 50 values
SpikeCount[:50]

In [ ]:
# make sure it worked
assert(SpikeCount[24]==2)

Now **compute the auto-correlation of the signal** using the function `correlate` from the `scipy` package.
We need to compute the measure on **centered data** (i.e. removing the mean of the value to the spike count data) so that the null values correspond to 0.

In [ ]:
# center the spike count data, i.e. remove its mean value
SpikeCountCentered = ??

# compute the auto-correlation (i.e. cross-correlation between the two same signals)
AC = correlate(???)

# this provides the values of the corresponding lags
lags = correlation_lags(len(SpikeCountCentered), len(SpikeCountCentered))

# lags are given in time steps, convert to seconds
lags = ??

# plot
plt.plot(???);
plt.xlabel(??);
plt.ylabel(??);

It's nice to have made it, but we can't really see anything because the auto-correlation covers the whole range of timing in the data (up to 4000 seconds, i.e. over an hour of recording). In reality we don't expect the most significant dynamics to last more than one second, so let's **plot the auto-correlation only for lags up to one second**.

Moreover, the peak corresponds to the value at exactly lag-zero, which is very not informative because at lag-zero one variable is always maximally correlated to each other. So we will **change this value to a nan** (use `np.nan`) to avoid distorting the scale along the y-axis.

In [ ]:
# define maximal lag
MaxLag = ??

# set value at lag 0 to nan (we need to convert first to floater)
AC = np.float64(AC)
???

# boolean array for all lags smaller than maximal lag (in absolute value)
mask = ???

# plot the AC again
plt.plot(???);
plt.xlabel(??);
plt.ylabel(??);

**Interpret the plot.**

## 2.2 Cross-correlation

Auto-correlation is like cross-correlation applied to the same signal, but what is cross-correlation? Cross-correlation allows to look for **interactions between two signals not only at corresponding times, but also with certain time lags**. For example, if one neuron receives excitatory synaptic input from another neuron, on general that higher activity in the pre-synaptic neuron will be followed with higher activity in the post-synaptic neuron (with a certain lag set by synaptic dynamics). This will be directly visible in the cross-correlogram as a bump occurring at the corresponding lag.
**Warning!** while it is tempting to interpret a cross-correlogram as a causal interaction between the two signals, cross-correlograms can also signal shared influence impacting both signals. In fact in neural activity, the probability that two neurons are connected by a direct synapse is rather low, so more often than not a cross-correlogram reveals the influence from shared incoming neural signals.

Load the spiking activity from neuron 4 and **compute the cross-correlogram between the two neurons**.
We will first **define a function to compute and plot the cross-correlogram**.

In [ ]:
# First, define a function to plot the cross-correlation between two signals
def plot_crosscorrelation(X, Y, dt, MaxLag):
    """
    Plots the cross-correlation between two signals 
    Args:
       X (array): first signal (time series)
       Y (array): second signal (time series)
       dt (float): time resolution (in sec)
       MaxLag (float): maximum lag to be plotted (in sec)
    """
    # center the two signals X and Y
    X =?? 
    Y = ??
    
    # compute the cross-correlation and lags
    CC = correlate(??)
    lags = correlation_lags(len(X), len(Y))
    
    # convert lags to seconds
    ???
    
    # define mask for lags below value set by MaxLag
    ???

    # plot
    ?????

In [ ]:
# Load data from neuron 4
SpikeTimes4 =  np.loadtxt("Neuron4_Spike.csv")

# Bin spikes (using same time grid as for neuron 1)
SpikeCount4 = ????

# use function created above to plot the cross-correlation
?????

**Interpret.**

## 2.3 (Optional) Cross-correlation on synthetic data
To understand better how cross-correlation, let's use it on simulated data. 
First let us simulate one neuron that spikes as a Poisson process with the same rate as neuron 1 in our experimental data.

**Simulate the spike counts in each bin for this synthetic neuron.** A Poisson process can be simulated using `np.random.poisson`.

In [ ]:
# average spike count across time bins
AverageCount = ??

# generate an array of spike count following Poisson process with corresponding rate
SyntheticSpike = np.random.poisson(??, ??)

# display first 100 bins
SyntheticSpike[:100]

**Now simulate the activity of a second (extremely over-simplified!) neuron receiving input from the first neuron and which outputs the exact same spike train as the input with a certain delay of 30 ms (so 3 time bins).**

In [ ]:
# Define lag (in time bins)
lag = ??

# Define activity of second neuron (Hint: we just need to add zeros at the beginning of the spike count array, and remove the corresponding values at the end)
SyntheticSpike2 = ???

# display first 100 bins (check that indeed spikes in neuron 1 are lagged in neuron 3 by 3 bins)
SyntheticSpike2[:100]

**Now plot the cross-correlation between the spike counts of the two neurons** (using our previously defined function).

In [ ]:
# Use  function
???

**Vary the lag and compute the cross-correlation again. Interpret.**

# 3. Rastergram and PSTH


## 3.1 Grouping spiking activity per trial

It is now time to start **relating spiking activity to the behavior**.
First, we need to extract the spiking activity corresponding to each trial. 
More precisely, on each trial, we want to **identify all the spikes in a certain window around the onset of the animal response and re-reference the spike times w.r.t. this response onset** (i.e. t=0 now corresponds to response onset). We will use a window from 500 ms prior to response onset to 1000 ms after response onset.
**Add the spike train per trial as a new variable to our dataframe** (under the name `Spikes`).

In [ ]:
# define window
window = ??

# number of trials
nTrial = ??

# initiliaze a list with the spikes corresponding to each trial
SpikesPerTrial = []

#loop through trials
for t in range(nTrial):
    
    # re-reference time so that time=0 corresponds to tResponse in this trial
    SpikeRereferenced = ???
    
    # select only spikes in window
    ????
    
    # add to SpikesPerTrial
    ????

# add to the dataframe
???

#check
df.head()

## 3.2 Rastergram
Now we can see how the activity of the neuron evolves within each trial by plotting a rastergram (or rasterplot).
In a rastergram, x-axis denotes time, y-axis denotes the different trials, and each spike is noted by a vertical tick (or point). A rastergram can also be used to plot the activity of a population if neurons across a single trial, and so the vertical axis denotes neuron identity.
**Plot the rastergram for this neuron.**
Add a vertical line to mark response onset.

In [ ]:
# loop through trials
for ????
    
    # number of spikes in corresponding window    
    nSpikes = ???
    
    # plot spike as blue vertical ticks
    plt.plot(??, ??,'b|');

# add line at response onset
plt.axvline(??);

# add labels
???

**Interpret.**

We see a lot of variability of neuron firing across trials. But importantly, in many trials we also see an increase of activity starting 200 ms approx after motion onset (although that increase does not seem to be present in all trials).

## 3.3 Rastergram per condition
The rastergram can also be separated by behavioral condition, to inspect visually whether the neural activity is related to some behavioral condition. 
**Plot the PSTH separately for trials with leftward and rightwards responses.**
(In practice, we use a different color for each type of response and we plot  trials in one condition above trials in the other condition, see an example [here](https://i.gyazo.com/06659ec5318b06dd3d82ef2fbd15948c.png) ).

In [ ]:
# identify trials with leftwards response (i.e. =1)
TrialIndexResp1 = ???

# number of leftward responses
nResp1 = ???

# loop through leftward response trials
for t in ??:
    # corresponding index of trial in list of ALL trials
    tt = ??
    
    # plot spikes for this trial (in blue)
    nSpikes = ??
    plt.plot(???);


# repeat the same for trials with rightwards responses (do not forget to change the color and add a vertical offset)
?????????????????????????????????    

    
# add line at response onset
???

**Interpret.**

## 3.4 PSTH
We can confirm our visual impression of rastergrams by plotting the Peri-Stimulus Time Histogram (or PSTH), which is an ugly name to say that we basically average the spike counts across trials, after binning the spikes using a certain window. By averaging activity across trials, we also gain statistical power to detect consistent changes in activity at a given time in a given condition.

First, **define a function that computes the PSTH for a given array of spike trains**. For this analysis we don't care about the trial identity of each spike, only their timing, so we can concatenate spike trains across trials using `np.concatenate`.

In [ ]:
# This function will compute the PSTH of a given array of spike trains
def compute_PSTH(S, dt, Tini, Tend):
    """
    Computes the PSTH. 
    Args:
       S (list of ndarrays): list of spike trains (one array per trial)
       dt (float): time resolution for PSTH
       Tini (float): initial time of PSTH
       Tend (float): final time of PSTH
    Returns:
       ndarray: PSTH
       ndarray: corresponding time bin centers
    """
    
    # number of trials in the array
    nTrial = ??
    
    # concatenate all spikes in one single array
    all_spike = ???
        
    # define the time grid, starting at Tini until Tend with dt resolution
    time_grid = ??

    # compute the spike counts in all bins defined by the time grid
    H = ???
    
    # the first element is the PSTH, the second is the time grid
    PSTH = H[0]
    
    # normalize the counts by number of trials and size of time bin so that PSTH is defined in spikes/sec
    norm_factor = ??
    PSTH = PSTH / norm_factor

    # remove last element in time grid to match length with PSTH
    time_grid = ??
    
    return PSTH, time_grid

**Plot the PSTH separately for leftward and rightward trials** (as two curves on the same plot).

In [ ]:
# time resolution
dt = .01

# spike trains for leftward and rightward response trials
# we need '.values' to convert dataframe back to array
Spikes_resp1 = df.??[??].values
Spikes_resp2 = df.??[??].values

# compute PSTH using the function above
???????

# plot
plt.plot(???????, label = "left");
plt.plot(??????, label = "right");

# vertical line
???

# add labels
???
plt.legend();

**Interpret**.

## 3.5 Smooth PSTH (Optional)
Because very fast changes in the spiking activity are often irrelevant, **PSTHs are often smoothed by convolving it with a gaussian kernel**. The width of the gaussian controls the time scale of smoothing. The functions below define a gaussian kernel and smooth a signal using this gaussian kernel.

In [ ]:
def gaussian_kernel(sigma):
    """
    Defines a gaussian kernel. 
    Args:
       sigma (float): width of gaussian kernel.
    Returns:
       ndarray: gaussian kernel
    """
    
    #define the length of the kernel
    kernel_length = 5*sigma
    
    # define indices for the kernel
    time_vec = np.arange(-3*sigma,3*sigma) 

    #generate a Gaussian kernel (a value for each index)
    K = norm.pdf(time_vec, loc=0, scale=sigma)
    
    #Normalize the kernel to have area=1 to maintain the same rate units
    K = K/np.sum(K)
    
    return K

    
def smooth_function(X, dt, sigma):
    """
    Smooths any time series with a gaussian kernel. 
    Args:
       X (ndarray): time series.
       dt (float): time resolution of time series
       sigma (float): width of gaussian kernel.
    Returns:
       ndarray: Smoothed time series
    """
    
    # convert sigma of kernel to time bins
    sigma_timebins = int(sigma/dt)

    # define gaussian kernel
    K = gaussian_kernel(sigma_timebins)
    
    # convolve signal with kernel
    Xsmooth = np.convolve(X, K, 'same')

    return Xsmooth

This function simply plots the gaussian kernel, so you get an idea.

In [ ]:
sigma = 10 # here define in time bins

plt.plot(gaussian_kernel(sigma));

Now **apply the function to smooth the PSTHs**. Choose an appropriate value for sigma.

In [ ]:
 #define the temporal width (in SECONDS) of the Gaussian kernel to filter the PSTH
sigma_kernel = ??

# use the function above to convolve the two PSTHS
SmoothPSTH_Resp1 = ??
SmoothPSTH_Resp2 = ??

# plot the smoothed PSTHs
?????????

# vertical line
????

# labels
??????
plt.legend();

# 4. Fano Factor

The Fano Factor is a **measure of the reliability of the spiking process**. In other words it tells us how stochastic spiking is. It is computed on spike counts on a window long enough to have on average more than one spike. The value is $FF = \frac{V}{M}$, where $V$ is the variance of the spike count across trials (windows), and $M$ is the mean of the spike count across trials.The homogeneous Poisson process has a Fano Factor equal to 1. Larger values means even more random than that (i.e. highly bursting), lower values means more regular firing (i.e. periodic).
The Fano Factor must be computed after taking into account the factors that we know affect the activity of the neuron (and so introduce variability in the spiking). Which is why we will compute separately for leftward and rightward responses.

**Compute the number of spikes in each trial in a window of 500 ms starting at response onset**, and as a variable to the dataframe.

In [ ]:
# Define window
MotorWindow = ??

# Pre-allocate spike count array
SC = np.zeros(??)

#loop through trials
for ????
    # number of spike counts within this window for this trial
    SC[t] = ????????

# add to dataframe
??

# check
df.head()

**Compute separately for leftward and rightward responses the average spike count, variance of the spike count and Fano Factor. Plot all values in bar plots.** (Note: the variance is computed using `np.var`.

In [ ]:
# pre-allocate numpy arrays for mean, variance of spike counts and Fano Factor
MeanCount = np.zeros(2)
VarCount = np.zeros(2)
FanoFactor  = np.zeros(2)

# for each type of response (leftward/rightward)
for r in range(2):
    
    # corresponding trials (mind the values taken by df.Response!)
    mask = ???
    
    # corresponding spike counts
    SpikeCountResp = ??
    
    # mean values
    MeanCount[r] = ??
    
    # variance
    VarCount[r] = ??
    
    # Fano Factor
    FanoFactor[r] = ??

# reponse label
RespLabel = ('Left','Right')

# plot mean rate
plt.subplot(1,3,1)
plt.bar(????)
plt.ylabel(??);
plt.xlabel(??)

# plot variance
plt.subplot(1,3,2)
plt.bar(????)
plt.ylabel(??);
plt.xlabel(??)

# plot Fano Factor
plt.subplot(1,3,3)
plt.bar(????)
plt.ylabel(??);
plt.xlabel(??)

plt.tight_layout()

**Interpret.**

# 5. Spike count regression

In this last part, we will see how we can perform **regression on spike counts data to determine the variables that a neuron is encoding**. This will be done using **Poisson regression** or **Poisson GLM**, which is the standard regression for count data (just as logistic regression is for binary data).

## 5.1 Logistic regression

But first, let us do look at how the animal behaves during this session. We have copied below (and adapted, thank you) the functions used in Assignments 2-3 to plot and fit a psychometric curve. Read through them if you need a little refresh on logistic regression.

In [ ]:
def logistic(x):
    """
    Returns the output of the logistic function for the given input value (float or array-like).
    """
    y = 1 / (1 + np.exp(-x))
    return y

def psychometric_model(d, w0, w1):
    """
    Compute the psychometric function based on a simple logistic model. 
    Args:
       d (ndarray): input values.
       w0 (float): intercept for logistic regression.
       w1 (float): slope for logistic regression.
    Returns:
       ndarray: The `y` data points of the psychometric function. 
          In our case, this corresponds to the probability of CCW responses.
    """
    p = logistic(w0 + w1*d)
    return p

def plotcurve(df, color):
    """
    Plot the fitted psychometric curve with experimental datapoints on top. 
    Args:
       df (dataframe): experimental data
       color (string): color of the datapoints and fitted line
    """

    mod = smf.glm(formula='Response ~ Stimulus', data=df, family=sm.families.Binomial())
    res = mod.fit()
    
    myx = np.linspace(-2,2,100)
    yfit=res.predict(pd.DataFrame({'Stimulus': myx})) #yfit = res.predict(exog={'probe_target':myx})

    # plot the psychometric function (fit)
    plt.plot(myx,yfit,'-', color=color,label='fit')
    
    # plot the psychometric curve (datapoints)
    df.groupby('Stimulus').Response.agg(('mean','sem')).plot(yerr='sem', color=color, fmt = 'o', ax=plt.gca(), label='data');

Before applying logistic regression, we need to make sure that our dependent variable (here the response) is coded as 0s and 1s. Now this is not the case. **Change the values so that 1s mark rightward responses.**

In [ ]:
# subtract 1, so 2 becomes 1 (right) and 1 becomes 0 (left)
?????

#check
df.head()

Now **use the functions above to plot the psychometric curve.**

In [ ]:
plotcurve(???)

plt.xlabel('stimulus evidence');
plt.ylabel('p(rightward response)');

**Interpret.**

## 5.2 Simple Poisson regression
Now we are ready for a simple regression of the neuron spike count (in the 500-ms response window).
**Regress the spike counts against stimulus evidence using a Poisson GLM**. In practice, this is done just like logistic regression using `glm` from `statsmodel` package, but setting the family to `Poisson` instead of `Binomial`.

In [ ]:
mod = smf.glm(formula=???, data=??, family=sm.families.Poisson())
res = mod.fit()
print(res.summary())

**Interpret.**

## 5.3 Multiple Poisson regression
The previous model only looked at the possible influene of stimulus onto spiking, but as for other GLMs we can add more factors in by changing the formula.
**Add a regressor for the response in the GLM.**

In [ ]:
mod = ???
res = ????
print(res.summary())

**Interpret the weights for the response. What about the stimulus weight?**

## 5.4 (Optional) Validation a Poisson regression model
Finally, as for any statistical model, it is important after fitting the model to validate it by checking if the behavior it predicts deviates or not from what is observed. This is equivalent to validating a simple logistic regression model by comparing the experimental and and fitted psychometric curves.

In Poisson regression, we can check whether the spike counts follow an exponential distribution, when experimental variables are controlled. Here, since stimulus plays no role in the spiking activity, we can **plot the distribution of spike counts separately for leftward and rightward responses against the predictions from the Poisson model**. The Poisson model predicts exponential distributions with a scale defined by the rate of the model.

Reminder: in the Poisson GLM, the rate (or expected value) is the exponential of the weighted sum of the regressors (use `np.exp`). The exponential is the non-linearity of the Poisson GLM, as the logistic function is non-linearity of logistic regression.

In [ ]:
ExpectedCount_Response = np.zeros(2)

# expected value of counts for leftward responses
ExpectedCount_Response[0] = ??

# expected value of counts for rightward responses
ExpectedCount_Response[1] = ??

# loop through left/right responses
for r in ??:
    # corresponding subplot
    plt.subplot(1,2,r+1)
    
    # select corresponding trials
    mask = ??
    
    #corresponding spike counts
    SpikeCountResp = ??
    
    # plot experimental distribution of counts
    ???
    
    # range of values to plot for exponential distribution
    xx = range(20)
    
    # scale is inverse of expected count
    scale = ??
    
    #probability of count according to Poisson distribution 
    pPoisson = expon.pdf(???)
    
    # plot Poisson prediction
    ???
    
    plt.xlabel('spike count');
    if r==0: plt.ylabel('density')  
    plt.legend();
    plt.title(RespLabel[r]);

**Interpret.**